In [1]:
%run read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bd148de1-831c-48a9-93c0-e0fcc6a7fcad;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (44ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (25ms)
:: resolution report :: resolve 646ms :: artifacts dl 73ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/03/26 23:04:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/26 23:04:33 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [53]:
%run trace_encode.ipynb

In [ ]:
#first question I have is how many events do I have when I call readfirst
df = readFirstEvents()

In [ ]:
#what is our count
df.count()

In [ ]:
#how many days do we have
df.select('event_day').distinct().show()

In [ ]:
#how many hours
df.select('event_hour').distinct().show()

In [ ]:
#how many malicious events do you have? 
df_m = df.where(col('malicious') == 1)
print("mal count: " +str(df_m.count()))
#how many benign events do you have? 
df_b = df.where(col('malicious') == 0)
print("benign count: " +str(df_b.count()))

In [ ]:
#compare proportions with original work, orig proportion is .0016%
print('mal_prop = ' + str((8596/4587096)*100))


In [ ]:
#so the prop is higher than the orig. We can live with that, how many mal events are on each day
df_m.groupBy('event_day').count().show()#.select('event_day').agg('count').distinct().count()
#I see these counts as generally in line with what the ANUBIS authors found.

In [ ]:
df_b.groupBy('event_day').count().show()

In [ ]:
#function to generate fake data
from pyspark.sql.types import StringType
import random

def gen_dummy_data(out,size):
    start_time = time.time()
    
    if out == 'b':
        # Define a function to generate binary strings with length 25
        def generate_binary_string():
            return format(random.randint(54360,33554432),'b').zfill(25)
    else:
        def generate_binary_string():
            return format(random.randint(1,10000000),'b').zfill(25)
        
    # Create a list of column names
    columns = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6']
    
    # Create a list of row data, each containing six binary strings
    data = [[generate_binary_string() for i in range(6)] for j in range(size)]

    # Create a PySpark dataframe from the row data and column names
    df = spark.createDataFrame(data, columns)
    
    #create an array of all the traces, combine in one column, and drop all the other
    #columns
    df = df.select(array('col1', 'col2',
                           'col3','col4', 
                           'col5', 'col6').alias('event_sequence'))
    
    #add the right malicious column
    if out == 'b':
        df = df.withColumn("malicious", lit(0))
        df = df.select("malicious", 'event_sequence')
        
    else: 
        df = df.withColumn("malicious", lit(1))
        df = df.select("malicious", 'event_sequence')
    
    #add a trace column in the front of the df
    df = df.withColumn("Trace", (monotonically_increasing_id() + 1))
    df = df.select("Trace", "malicious", 'event_sequence')
    
    print("elapsed: "+ str(time.time() - start_time))
    
    return df


    

In [ ]:
df_samp_b = gen_dummy_data('b',1000)
df_samp_b.limit(10).show()
test = df_samp_b.limit(10).select('event_sequence').collect()
print(len(test[0][0]))
print(len(test[0][0][3]))
df_samp_b.unpersist()

In [ ]:
#now generate all the real data
df_samp_b500k = gen_dummy_data('b',500000)

In [ ]:
df_samp_brest = gen_dummy_data('b',2091594)

In [ ]:
df_samp_m = gen_dummy_data('m',6517)
df_samp_m.limit(3).show()

In [ ]:
#union it all and then write it to parquet
df_samp_b_all = df_samp_b500k.union(df_samp_brest)
#show me some sample rows to be sure
df_samp_b_all.limit(3).show()

In [ ]:
df_samp_all = df_samp_b_all.union(df_samp_m)

In [ ]:
#now write the first set to parquet as a train test dataset
df_samp_all.write.option("maxRecordsPerFile", 300000).mode("overwrite")\
.parquet(f'{s3_url_trusted}/prod/tensor_sample_data/train_val')

In [ ]:
#now we do this all over again for the holdout set
df_b = gen_dummy_data('b',1347693)
df_m = gen_dummy_data('m', 459)
df_all = df_b.union(df_m)
df_samp_all.write.option("maxRecordsPerFile", 300000).mode("overwrite")\
.parquet(f'{s3_url_trusted}/prod/tensor_sample_data/test_holdout')

In [ ]:
#does first events work now on data duplication
cg = create_graph(df_m)

In [ ]:
cg.limit(10).toPandas()
#answer I belive it did. 

In [23]:
#exploration of data read in under the first events folder
#first read in the data, filtered only on 23Sep
df = checkFirstEventRead()

12:39AM UTC on Mar 27, 2023 --- read time: 3.1958396434783936 seconds ---


In [ ]:
df.printSchema()

In [ ]:
df.select('event_day').distinct().show()
df.select('event_hour').distinct().show()

In [4]:
df_24 = checkFirstEventRead(24)
df_25 = checkFirstEventRead(25)

 4:10PM UTC on Mar 26, 2023 --- read time: 0.3442869186401367 seconds ---
 4:10PM UTC on Mar 26, 2023 --- read time: 0.2903435230255127 seconds ---


In [ ]:
df_24.select('event_day','event_hour').distinct().show()
df_25.select('event_day','event_hour').distinct().show()

In [ ]:
print(df.count())
print(df_24.count())
print(df_25.count())

In [ ]:
#count how many hosts in each
print(df.select('hostname').distinct().count())
print(df_24.select('hostname').distinct().count())
print(df_25.select('hostname').distinct().count())

In [ ]:
#malicious event count
#count how many hosts in each
print(df.where(col('malicious')==1).count())
print(df_24.where(col('malicious')==1).count())
print(df_25.where(col('malicious')==1).count())

In [24]:
#now let's create file extension columns
df_23 = df.withColumn("image_path_ext", regexp_extract("image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("parent_path_ext", regexp_extract("parent_image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("new_path_ext", regexp_extract("new_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("file_path_ext", regexp_extract("file_path", "\.[0-9a-z]+$", 0))

In [ ]:
df_23.printSchema()

In [ ]:
df_23.select('file_path_ext').distinct().show()

In [ ]:
df_23.select('id').distinct().collect()[0][0]

In [ ]:
df_23.select('file_path').distinct().collect()[:20]

In [ ]:
df_23.select('file_path_ext').collect()[:20]

In [ ]:
#how many distinct counts are there
print(df_23.select('file_path').distinct().count())
print(df_23.select('file_path_ext').distinct().count())
#image path
print(df_23.select('image_path').distinct().count())
print(df_23.select('image_path_ext').distinct().count())
#parent image path
print(df_23.select('parent_image_path').distinct().count())
print(df_23.select('parent_path_ext').distinct().count())
#new path
print(df_23.select('new_path').distinct().count())
print(df_23.select('new_path_ext').distinct().count())

So I think I want to use iimage path extension, parent path, and then maybe new path for files.

In [6]:
#combine 23-24 and run all again
df_24 = df_24.withColumn("image_path_ext", regexp_extract("image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("parent_path_ext", regexp_extract("parent_image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("new_path_ext", regexp_extract("new_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("file_path_ext", regexp_extract("file_path", "\.[0-9a-z]+$", 0))

In [8]:
df_2324 = df_23.union(df_24)

In [7]:
#combine 23-24 and run all again
df_25 = df_25.withColumn("image_path_ext", regexp_extract("image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("parent_path_ext", regexp_extract("parent_image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("new_path_ext", regexp_extract("new_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("file_path_ext", regexp_extract("file_path", "\.[0-9a-z]+$", 0))

In [ ]:
#how many distinct counts are there for combined training
print(df_2324.select('file_path').distinct().count())
print(df_2324.select('file_path_ext').distinct().count())
#image path
print(df_2324.select('image_path').distinct().count())
print(df_2324.select('image_path_ext').distinct().count())
#parent image path
print(df_2324.select('parent_image_path').distinct().count())
print(df_2324.select('parent_path_ext').distinct().count())
#new path
print(df_2324.select('new_path').distinct().count())
print(df_2324.select('new_path_ext').distinct().count())

In [10]:
#create graph with 23 data and see what is going on
graph_23_m = df_23.where(col('malicious')==1)

In [11]:
graph_23_m.count()

3624

In [12]:
graph_23_m = create_graph(graph_23_m)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


event traces: 1261


In [29]:
def transp_expl(df_graph):
    #add "trace index" to keep track of traces. important for transposing back
    df_transp = df_graph.withColumn("Trace", (monotonically_increasing_id() + 1))
    df_transp = df_transp.select("Trace", 
                                 *[col for col in df_transp.columns if col != "Trace"])

    #drop all vertices
    df_transp = df_transp.drop('a','b','c','d','e','f','g')

    #transpose rows 
    stacked_df = df_transp.selectExpr(
        "Trace", 
        "posexplode(array(e1, e2, e3, e4, e5, e6)) as (pos, col)"
    ).select(
        "Trace", 
        expr('''CASE pos 
        WHEN 0 THEN 'e1' 
        WHEN 1 THEN 'e2'
        WHEN 2 THEN 'e3'
        WHEN 3 THEN 'e4'
        WHEN 4 THEN 'e5'
        ELSE 'e6' END''').alias("event"),
        "col"
    ).orderBy("Trace","event")

    #explode columns
    df_onehot = stacked_df.select(*stacked_df.columns, "col.*").drop('col')
    
    return df_onehot

In [13]:
graph_exp = transp_expl(graph_23_m)

In [15]:
graph_exp.limit(24).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,image_path,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious
0,1,e1,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:25:52.486,PROCESS,CREATE,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1
1,1,e2,a52f6519-6265-4437-9f6c-3402fb245cc7,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,2019-09-23 15:40:21.404,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1
2,1,e3,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:25:52.486,PROCESS,CREATE,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1
3,1,e4,a52f6519-6265-4437-9f6c-3402fb245cc7,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,2019-09-23 15:40:21.404,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1
4,1,e5,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:25:52.486,PROCESS,CREATE,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1
5,1,e6,a52f6519-6265-4437-9f6c-3402fb245cc7,36d062d4-1f69-4dcc-9791-46b5c07dadef,2019-09-23 17:21:39.921,FILE,READ,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,\Device\HarddiskVolume1\Windows\system32\Windo...,None,None,None,None,1
6,2,e1,4aec5900-4572-4d09-81e6-fbc227a7f58d,3eea9071-9ceb-468a-ad89-9e3890833e52,2019-09-23 17:34:17.240,PROCESS,CREATE,SysClient0660.systemia.com,None,None,powershell.exe,\Device\HarddiskVolume1\Windows\system32\wbem\...,None,None,None,None,None,None,1
7,2,e2,3eea9071-9ceb-468a-ad89-9e3890833e52,4aec5900-4572-4d09-81e6-fbc227a7f58d,2019-09-23 17:43:13.564,PROCESS,OPEN,SysClient0660.systemia.com,None,None,powershell.exe,powershell.exe,None,None,None,None,None,None,1
8,2,e3,4aec5900-4572-4d09-81e6-fbc227a7f58d,3eea9071-9ceb-468a-ad89-9e3890833e52,2019-09-23 17:34:17.240,PROCESS,CREATE,SysClient0660.systemia.com,None,None,powershell.exe,\Device\HarddiskVolume1\Windows\system32\wbem\...,None,None,None,None,None,None,1
9,2,e4,3eea9071-9ceb-468a-ad89-9e3890833e52,4aec5900-4572-4d09-81e6-fbc227a7f58d,2019-09-23 17:43:13.564,PROCESS,OPEN,SysClient0660.systemia.com,None,None,powershell.exe,powershell.exe,None,None,None,None,None,None,1


In [17]:
#try to weed out bidirectional events
df_23_m = df_23.where(col('malicious')==1)

In [62]:
# create a sample dataframe that aligns closer with what you see above. 
#I think the trick is going to be partitioning based on object action and timestamp.
#if those three things are the same then pull the first one out
data = [("A", "B", 1,'process','create'), 
        ("A", "B", 1,'process','create'), 
        ("B", "A", 2,'process','open'),
        ("B", "A" ,2,'process','open'),
        ("B", "C", 3,'file','read'),
       ("D", "E", 4,'process','create'), 
        ("D", "E", 4,'process','create'), 
        ("E", "D", 5,'process','open'),
        ("E", "D" ,5,'process','open'),
        ("E", "F", 6,'file','read')]
columns = ["src", "dst", "time","object","action"]
df = spark.createDataFrame(data, columns)

In [63]:
df.show()

+---+---+----+-------+------+
|src|dst|time| object|action|
+---+---+----+-------+------+
|  A|  B|   1|process|create|
|  A|  B|   1|process|create|
|  B|  A|   2|process|  open|
|  B|  A|   2|process|  open|
|  B|  C|   3|   file|  read|
|  D|  E|   4|process|create|
|  D|  E|   4|process|create|
|  E|  D|   5|process|  open|
|  E|  D|   5|process|  open|
|  E|  F|   6|   file|  read|
+---+---+----+-------+------+



In [65]:
window = Window.partitionBy("sorted").orderBy("time")
df_2 = df.withColumn('sorted', sort_array(array(col('src'),col('dst')) )) \
                .withColumn('rank', rank().over(window)) \
                .filter(col('rank') == 1) \
                .drop('rank')
df_2.show()

+---+---+----+-------+------+------+
|src|dst|time| object|action|sorted|
+---+---+----+-------+------+------+
|  A|  B|   1|process|create|[A, B]|
|  A|  B|   1|process|create|[A, B]|
|  B|  C|   3|   file|  read|[B, C]|
|  D|  E|   4|process|create|[D, E]|
|  D|  E|   4|process|create|[D, E]|
|  E|  F|   6|   file|  read|[E, F]|
+---+---+----+-------+------+------+



In [18]:
#try array sort
df_4 = df.withColumn('sorted',sort_array(array(col('src'),col('dst'))))

In [19]:
df_4.show()

+---+---+----+-------+------+------+
|src|dst|time| object|action|sorted|
+---+---+----+-------+------+------+
|  A|  B|   1|process|create|[A, B]|
|  A|  B|   1|process|create|[A, B]|
|  B|  A|   2|process|  open|[A, B]|
|  B|  A|   2|process|  open|[A, B]|
|  B|  C|   3|   file|  read|[B, C]|
|  D|  E|   4|process|create|[D, E]|
|  D|  E|   4|process|create|[D, E]|
|  E|  D|   5|process|  open|[D, E]|
|  E|  D|   5|process|  open|[D, E]|
|  E|  F|   6|   file|  read|[E, F]|
+---+---+----+-------+------+------+



In [20]:
window = Window.partitionBy("sorted").orderBy("time")
df_5 = df_4.withColumn("row_num", row_number().over(window))
df_5.show()

+---+---+----+-------+------+------+-------+
|src|dst|time| object|action|sorted|row_num|
+---+---+----+-------+------+------+-------+
|  A|  B|   1|process|create|[A, B]|      1|
|  A|  B|   1|process|create|[A, B]|      2|
|  B|  A|   2|process|  open|[A, B]|      3|
|  B|  A|   2|process|  open|[A, B]|      4|
|  B|  C|   3|   file|  read|[B, C]|      1|
|  D|  E|   4|process|create|[D, E]|      1|
|  D|  E|   4|process|create|[D, E]|      2|
|  E|  D|   5|process|  open|[D, E]|      3|
|  E|  D|   5|process|  open|[D, E]|      4|
|  E|  F|   6|   file|  read|[E, F]|      1|
+---+---+----+-------+------+------+-------+



In [22]:
df_6 = df_5.filter(df_5.row_num == 1)
df_6 = df_6.drop('row_num')
df_6.show()

+---+---+----+-------+------+------+
|src|dst|time| object|action|sorted|
+---+---+----+-------+------+------+
|  A|  B|   1|process|create|[A, B]|
|  B|  C|   3|   file|  read|[B, C]|
|  D|  E|   4|process|create|[D, E]|
|  E|  F|   6|   file|  read|[E, F]|
+---+---+----+-------+------+------+



In [77]:
#now let's apply the above to the mal matrix
#run read first events of df_23, count should be the same
print(df_23.count())

def getFirstEvents(df):
    """
    this creates then drops duplicates and gets the first appearance of each relationship entry
    input - dataframe with columns objectID and actorID
    output dataframe
    """
    window = Window.partitionBy("relationship").orderBy("timestamp")
    # label malicious events, filter to only hosts with malicious events
    df = df.withColumn('relationship', concat(df.actorID, lit('->'),df.objectID) ) \
                .withColumn('rank', rank().over(window)) \ 
                .filter(col('row_num') == 1) \
                .drop('row_num')
    mal_hosts = list(df.select('hostname').filter( col('malicious') == 1 ).distinct().toPandas()['hostname'])
    df = df.filter(col("hostname").isin(mal_hosts))
    return df

df_23 = getFirstEvents(df_23)

print(df_23.count())

948334


804424


In [78]:
#create graph with 23 data and see what is going on
df_23_m_1 = df_23.where(col('malicious')==1)
df_23_m_1.count()

3320

In [54]:
#create first graph
graph_23_m_1 = create_graph(df_23_m_1)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


event traces: 6978


In [55]:
trace_m_1 = transp_expl(graph_23_m_1)

In [56]:
#now show what you have
trace_m_1.limit(24).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,image_path,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious
0,1,e1,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
1,1,e2,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
2,1,e3,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
3,1,e4,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
4,1,e5,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
5,1,e6,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
6,2,e1,28a6220d-2b91-4700-982f-1643cca55d39,28a6220d-2b91-4700-982f-1643cca55d39,2019-09-23 17:29:25.233,PROCESS,OPEN,SysClient0402.systemia.com,None,None,powershell.exe,powershell.exe,None,None,None,None,None,None,1
7,2,e2,28a6220d-2b91-4700-982f-1643cca55d39,28a6220d-2b91-4700-982f-1643cca55d39,2019-09-23 17:29:25.233,PROCESS,OPEN,SysClient0402.systemia.com,None,None,powershell.exe,powershell.exe,None,None,None,None,None,None,1
8,2,e3,28a6220d-2b91-4700-982f-1643cca55d39,28a6220d-2b91-4700-982f-1643cca55d39,2019-09-23 17:29:25.233,PROCESS,OPEN,SysClient0402.systemia.com,None,None,powershell.exe,powershell.exe,None,None,None,None,None,None,1
9,2,e4,28a6220d-2b91-4700-982f-1643cca55d39,28a6220d-2b91-4700-982f-1643cca55d39,2019-09-23 17:29:25.233,PROCESS,OPEN,SysClient0402.systemia.com,None,None,powershell.exe,powershell.exe,None,None,None,None,None,None,1


In [58]:
#attempt to sort the trace...that did not work
trace_m_1 = trace_m_1.withColumn('sorted',sort_array(array(col('src'),col('dst'))))
window = Window.partitionBy("sorted").orderBy("timestamp")
trace_m_2 = trace_m_1.withColumn("row_num", row_number().over(window))
trace_m_2 = trace_m_2.filter(trace_m_2.row_num == 1)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,image_path,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious
0,481036337153,e1,00e1deee-e034-4f15-8687-edf25b853f6d,00e1deee-e034-4f15-8687-edf25b853f6d,2019-09-23 15:40:26.568,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1
1,1013612281857,e1,013df866-fc1b-4c46-bc48-88c6ec4dbd5f,013df866-fc1b-4c46-bc48-88c6ec4dbd5f,2019-09-23 17:15:23.207,FILE,READ,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,\Device\HarddiskVolume1\Windows\system32\Windo...,None,None,None,None,1
2,1331439861761,e1,01b75004-b9b2-4be1-a70e-dc1fdde59fc5,01b75004-b9b2-4be1-a70e-dc1fdde59fc5,2019-09-23 18:45:04.105,SHELL,COMMAND,SysClient0419.systemia.com,None,None,powershell.exe,None,None,None,None,None,None,None,1
3,661424963586,e1,02572481-cb1c-42b2-8be4-35df8b9ed733,02572481-cb1c-42b2-8be4-35df8b9ed733,2019-09-23 18:45:16.837,FILE,CREATE,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\Users\Default\AppData\...,None,None,None,None,1
4,644245094401,e1,02874b0e-a7e1-4656-8015-d1b5467c7209,02874b0e-a7e1-4656-8015-d1b5467c7209,2019-09-23 17:21:38.714,FILE,READ,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,\Device\HarddiskVolume1\Windows\system32\CatRo...,None,None,None,None,1
5,68719476737,e1,029cb483-df9c-4613-a32d-9bcc872ded3e,029cb483-df9c-4613-a32d-9bcc872ded3e,2019-09-23 18:45:06.322,FILE,READ,SysClient0503.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\Windows\SYSTEM32\wbemc...,None,None,None,None,1
6,1477468749825,e1,02a005c7-6fd8-49a8-98d8-e0a4bdf942d8,02a005c7-6fd8-49a8-98d8-e0a4bdf942d8,2019-09-23 18:44:59.075,FILE,READ,SysClient0321.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\Windows\System32\AppLo...,None,None,None,None,1
7,257698037761,e1,03978db0-8f57-46b3-a6d8-0f4b9be44741,03978db0-8f57-46b3-a6d8-0f4b9be44741,2019-09-23 18:45:00.299,FILE,READ,SysClient0503.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\ASSEMBLY\NATIV...,None,None,None,None,1
8,1632087572481,e1,0487e062-1253-4b97-8eb4-6f2e5de22460,0487e062-1253-4b97-8eb4-6f2e5de22460,2019-09-23 18:45:04.518,FILE,READ,SysClient0462.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\Windows\system32\wbem\...,None,None,None,None,1
9,712964571137,e1,05c20b7f-317a-4405-abbb-c3a62547217f,05c20b7f-317a-4405-abbb-c3a62547217f,2019-09-23 17:29:38.765,FILE,READ,SysClient0402.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\Windows\system32\Windo...,None,None,None,None,1


In [60]:
trace_m_2 = trace_m_2.drop('row_num','sorted').sort('Trace')
trace_m_2.limit(24).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,image_path,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious
0,1,e1,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
1,2,e1,28a6220d-2b91-4700-982f-1643cca55d39,28a6220d-2b91-4700-982f-1643cca55d39,2019-09-23 17:29:25.233,PROCESS,OPEN,SysClient0402.systemia.com,None,None,powershell.exe,powershell.exe,None,None,None,None,None,None,1
2,3,e1,36d062d4-1f69-4dcc-9791-46b5c07dadef,36d062d4-1f69-4dcc-9791-46b5c07dadef,2019-09-23 17:21:39.921,FILE,READ,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,\Device\HarddiskVolume1\Windows\system32\Windo...,None,None,None,None,1
3,4,e1,44005d0c-6dc2-480a-b061-ebb46aa5a577,44005d0c-6dc2-480a-b061-ebb46aa5a577,2019-09-23 18:44:53.892,PROCESS,CREATE,SysClient0104.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\conho...,powershell.exe,None,None,None,None,None,None,1
4,68,e1,4b9755ad-05bb-458e-b9ff-27852eb51860,4b9755ad-05bb-458e-b9ff-27852eb51860,2019-09-23 17:55:57.368,FILE,READ,SysClient0660.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\Windows\system32\Windo...,None,None,None,None,1
5,69,e1,50ac366a-4500-4ef0-8579-22adacaf6438,50ac366a-4500-4ef0-8579-22adacaf6438,2019-09-23 17:40:59.882,FILE,READ,SysClient0660.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\Windows\System32\vault...,None,None,None,None,1
6,70,e1,526d2eb9-b1e1-46a5-9fdf-e205ec2c0301,526d2eb9-b1e1-46a5-9fdf-e205ec2c0301,2019-09-23 18:45:15.585,FILE,READ,SysClient0170.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\Windows\Microsoft.NET\...,None,None,None,None,1
7,71,e1,5df331a2-051f-4283-ae16-f9723c9366df,5df331a2-051f-4283-ae16-f9723c9366df,2019-09-23 18:45:05.149,FILE,READ,SysClient0771.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\Windows\SYSTEM32\secur...,None,None,None,None,1
8,72,e1,6baf37a8-f1d4-441d-aea3-27c98a859432,6baf37a8-f1d4-441d-aea3-27c98a859432,2019-09-23 17:29:26.235,PROCESS,CREATE,SysClient0402.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\PING.EXE,powershell.exe,None,None,None,None,None,None,1
9,73,e1,6fba2e2c-1552-41ad-80ee-fffdfbe58d29,6fba2e2c-1552-41ad-80ee-fffdfbe58d29,2019-09-23 17:43:42.454,SHELL,COMMAND,SysClient0660.systemia.com,None,None,powershell.exe,None,None,None,None,None,None,None,1


In [66]:
#de-dupe bi-d
print(df_23.count())
window = Window.partitionBy("sorted").orderBy("timestamp")
df_23_2b = df_23.withColumn('sorted', sort_array(array(col('objectID'),col('actorID')) )) \
                .withColumn('rank', rank().over(window)) \
                .filter(col('rank') == 1) \
                .drop('rank')
df_2.show()
#df_23 = df_23.withColumn('sorted',sort_array(array(col('objectID'),col('actorID'))))
#window = Window.partitionBy("sorted").orderBy("timestamp")
#df_23_2 = df_23.withColumn("row_num", row_number().over(window))
#df_23_2 = df_23_2.filter(df_23_2.row_num == 1)
#df_23_2 = df_23_2.drop('row_num','sorted')
print(df_23_2.count())
print(df_23_2b.count())

948334
+---+---+----+-------+------+------+
|src|dst|time| object|action|sorted|
+---+---+----+-------+------+------+
|  A|  B|   1|process|create|[A, B]|
|  A|  B|   1|process|create|[A, B]|
|  B|  C|   3|   file|  read|[B, C]|
|  D|  E|   4|process|create|[D, E]|
|  D|  E|   4|process|create|[D, E]|
|  E|  F|   6|   file|  read|[E, F]|
+---+---+----+-------+------+------+



798754


941132


In [67]:
#create graph with 23 data and see what is going on
df_23_m_2b = df_23_2b.where(col('malicious')==1)
print(df_23_m_2b.count())
graph_23_m_2b = create_graph(df_23_m_2b)
trace_m_2b = transp_expl(graph_23_m_2b)
#now show what you have
trace_m_2b.limit(6).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


3611


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


event traces: 69461


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,image_path,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious
0,1,e1,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
1,1,e2,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
2,1,e3,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
3,1,e4,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
4,1,e5,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1
5,1,e6,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,0fdc5360-36ae-4dd2-8f28-e09baa351d1f,2019-09-23 18:44:59.515,FILE,READ,SysClient0955.systemia.com,None,None,powershell.exe,None,None,\Device\HarddiskVolume1\WINDOWS\SYSTEM32\ATL.DLL,None,None,None,None,1


In [73]:
#de-dupe bi-d
print(df_23_m_1.count())
df_23_m_2 = df_23_m_1.withColumn('sorted',sort_array(array(col('objectID'),col('actorID'))))
window = Window.partitionBy("sorted").orderBy("timestamp")
df_23_m_2 = df_23_m_2.withColumn("row_num", row_number().over(window))
df_23_m_2 = df_23_m_2.filter(df_23_m_2.row_num != 1)
#df_23_m_2 = df_23_m_2.drop('row_num','sorted')
#print(df_23_m_2.count())

3624


In [43]:
print(df_23_m_2.count())

3318


In [44]:
graph_23_m_2 = create_graph(df_23_m_2)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


event traces: 0


In [45]:
df_23_m_2.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- relationship: string (nullable = true)
 |-- image_path_ext: string (nullable = true)
 |-- parent_path_ext: string

In [74]:
df_23_m_2.limit(20).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action,hostname,user_name,privileges,image_path,...,event_day,event_hour,malicious,relationship,image_path_ext,parent_path_ext,new_path_ext,file_path_ext,sorted,row_num
0,a2841c74-1db6-4c8b-8ad6-57cb44a77be0,2019-09-23 17:15:19.166,0179b33b-8882-40af-b2a6-71c04ec6e273,a52f6519-6265-4437-9f6c-3402fb245cc7,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,...,23,17,1,a52f6519-6265-4437-9f6c-3402fb245cc7->0179b33b...,.exe,.exe,None,None,"[0179b33b-8882-40af-b2a6-71c04ec6e273, a52f651...",2
1,75fbedc7-eaa0-4ef7-9a18-1305ee605ea0,2019-09-23 17:15:19.166,0179b33b-8882-40af-b2a6-71c04ec6e273,a52f6519-6265-4437-9f6c-3402fb245cc7,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,...,23,17,1,a52f6519-6265-4437-9f6c-3402fb245cc7->0179b33b...,.exe,.exe,None,None,"[0179b33b-8882-40af-b2a6-71c04ec6e273, a52f651...",3
2,3995a8e0-7cf5-4ea2-9de5-3f0971bf4af7,2019-09-23 17:15:17.125,01962cd7-0865-4877-a583-f7b1e200f54c,a52f6519-6265-4437-9f6c-3402fb245cc7,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,...,23,17,1,a52f6519-6265-4437-9f6c-3402fb245cc7->01962cd7...,.exe,.exe,None,None,"[01962cd7-0865-4877-a583-f7b1e200f54c, a52f651...",2
3,0de9f6c9-2beb-4372-bbf0-0cf331dc834f,2019-09-23 17:15:01.936,025eb157-4aa7-470f-919e-4854d5cbe163,a52f6519-6265-4437-9f6c-3402fb245cc7,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,...,23,17,1,a52f6519-6265-4437-9f6c-3402fb245cc7->025eb157...,.exe,.exe,None,None,"[025eb157-4aa7-470f-919e-4854d5cbe163, a52f651...",2
4,102d8c1b-b2c0-45e5-986f-c5c1b6172136,2019-09-23 17:15:00.330,03b0993f-45b9-4593-99f1-f1a9ae477bfe,a52f6519-6265-4437-9f6c-3402fb245cc7,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,...,23,17,1,a52f6519-6265-4437-9f6c-3402fb245cc7->03b0993f...,.exe,.exe,None,None,"[03b0993f-45b9-4593-99f1-f1a9ae477bfe, a52f651...",2
5,50da4523-9f32-45ab-bcc8-c5a5bfe7a695,2019-09-23 17:29:28.891,062ef785-dda0-4478-a972-d88220bf4dfc,e490633e-4e82-4bbc-b5bb-38f61777c4f7,PROCESS,OPEN,SysClient0402.systemia.com,None,None,powershell.exe,...,23,17,1,e490633e-4e82-4bbc-b5bb-38f61777c4f7->062ef785...,.exe,.exe,None,None,"[062ef785-dda0-4478-a972-d88220bf4dfc, e490633...",2
6,e9cb76c2-2437-4a80-a353-b5c898065148,2019-09-23 17:29:28.891,062ef785-dda0-4478-a972-d88220bf4dfc,e490633e-4e82-4bbc-b5bb-38f61777c4f7,PROCESS,OPEN,SysClient0402.systemia.com,None,None,powershell.exe,...,23,17,1,e490633e-4e82-4bbc-b5bb-38f61777c4f7->062ef785...,.exe,.exe,None,None,"[062ef785-dda0-4478-a972-d88220bf4dfc, e490633...",3
7,fd71f194-d61b-4e55-91da-774e0d7ae155,2019-09-23 17:15:09.931,07596e35-b55a-4a28-8b8e-a08b1e28b99c,a52f6519-6265-4437-9f6c-3402fb245cc7,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,...,23,17,1,a52f6519-6265-4437-9f6c-3402fb245cc7->07596e35...,.exe,.exe,None,None,"[07596e35-b55a-4a28-8b8e-a08b1e28b99c, a52f651...",2
8,0b22b8e7-b243-4b3d-bf69-dd3b9ef8832c,2019-09-23 17:15:16.887,0911473a-92da-495f-b267-3e70517e7621,a52f6519-6265-4437-9f6c-3402fb245cc7,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,...,23,17,1,a52f6519-6265-4437-9f6c-3402fb245cc7->0911473a...,.exe,.exe,None,None,"[0911473a-92da-495f-b267-3e70517e7621, a52f651...",2
9,9a152653-ba08-40b5-80a4-5994117b6899,2019-09-23 18:44:58.140,0ee8161a-8fd0-4e0e-a015-e8fb0e993575,64c0966e-66f8-452a-b8d9-ced3e2220a18,FILE,CREATE,SysClient0205.systemia.com,None,None,powershell.exe,...,23,18,1,64c0966e-66f8-452a-b8d9-ced3e2220a18->0ee8161a...,.exe,None,None,.psm1,"[0ee8161a-8fd0-4e0e-a015-e8fb0e993575, 64c0966...",2
